# AdaBoostRegressor

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
%matplotlib inline



In [24]:
df = pd.read_csv("dataset-encode-1.csv", sep = ";")

In [25]:
# TODO séparer dataset d'entrainement et dataset de test
y = df["Comptage horaire"]
X = df.drop(columns=["Comptage horaire"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
# Standardisation
col_norm = ["Jour", "Mois", "Année", "Heure", "Jour_semaine", "Jour férié", "Vacances scolaires"]

scaler = StandardScaler()
df[col_norm] = scaler.fit_transform(df[col_norm])

In [27]:
X_train_sample = X_train.sample(50000, random_state=42)
y_train_sample = y_train.loc[X_train_sample.index]

model = AdaBoostRegressor(
    estimator = DecisionTreeRegressor(max_depth = 60),  # Profondeur de l'arbre
    n_estimators = 45,        # Diminuer pour éviter l’overfitting
    learning_rate = 0.5,       # Réduction du taux d’apprentissage
    loss = "square",           # Fonction de perte plus stable
    random_state = 66
)


# Entraînement
model.fit(X_train_sample, y_train_sample)

# Prédictions
y_pred = model.predict(X_test)
y_pred = np.maximum(y_pred, 0) #Permet d'éviter les valeurs négatives
y_pred = np.round(y_pred) # Arrondi à l'entier le plus proche
# Comparaison des résultats
import pandas as pd
df_results = pd.DataFrame({"Réel": list(y_test), "Prédit": list(y_pred)})

In [21]:
display(df_results)

,Réel,Prédit
0,1,3.0
1,37,54.0
2,19,24.0
3,41,57.0
4,1,2.0
...,...,...
353724,3,5.0
353725,45,71.0
353726,33,31.0
353727,72,55.0


In [28]:
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

In [29]:
print(mse)
print(rmse)

2768.613461152464
52.61761550234355


In [30]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Calcul du R²
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

print(f"Score R² Entraînement : {r2_train}")
print(f"Score R² Test : {r2_test}")

Score R² Entraînement : 0.7515100244905416
Score R² Test : 0.7476518009407521
